In [1]:
#@title Mount your Google Drive
# If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# you can delete this cell which is specific to Google Colab. You may also
# change the paths for data/logs in Arguments below.
%matplotlib inline
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install transformers datasets evaluate rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 

In [3]:
import sys
import os
import shutil
import warnings
import pandas as pd
import numpy as np
import json
import torch
import evaluate

from transformers import (
  AutoModelForSeq2SeqLM, 
  DataCollatorForSeq2Seq,
  Seq2SeqTrainingArguments, 
  Seq2SeqTrainer,
  AutoTokenizer, 
  pipeline
) 

from pathlib import Path
from datasets import load_dataset, load_metric

In [4]:
# Paths

PROJ_DIR = Path('/content/gdrive/MyDrive/IFT6759/quick-recipe')
LOG_DIR = PROJ_DIR / 'logs'

LOG_DIR.mkdir(parents=True, exist_ok=True) 

if str(PROJ_DIR) not in sys.path:
    sys.path.insert(0, str(PROJ_DIR))

MODEL_DIR = PROJ_DIR / 'models'
DATA_DIR = PROJ_DIR / 'youcook2'
MODEL_SAVE_DIR = MODEL_DIR / 'youcook_t5_small'
LOG_SAVE_DIR = LOG_DIR / 'youcook_t5_small'

MODEL_SAVE_DIR.mkdir(parents=True, exist_ok=True) 
LOG_SAVE_DIR.mkdir(parents=True, exist_ok=True) 

ANNOTATED_DF_PATH = str(DATA_DIR / 'reviewed_0812.csv')
TRAIN_FRAC = 0.7
MAX_INPUT_LENGTH = 1024
MAX_SUMMARY_LENGTH = 128

RANDOM_SEED = 23456
np.random.seed(RANDOM_SEED)

In [ ]:
df = pd.read_csv(ANNOTATED_DF_PATH)

In [ ]:
key_sentences = df[df['IsUsefulSentence'] == 1]

In [ ]:
len(key_sentences)

3569

In [ ]:
indices = list(range(len(key_sentences)))
np.random.shuffle(indices)
train_len = int(TRAIN_FRAC * len(key_sentences))

In [ ]:
sentences, instructions = key_sentences['Sentence'].to_numpy() , key_sentences['Key steps'].to_numpy()

In [ ]:
train_sentences, train_instructions = sentences[:train_len], instructions[:train_len]
val_sentences, val_instructions = sentences[train_len:], instructions[train_len:]

In [ ]:
len(train_sentences), len(val_sentences)

(2498, 1071)

In [ ]:
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, model_max_length=MAX_INPUT_LENGTH)

In [ ]:
prefix = "summarize: "

def preprocess_function(examples, max_input_length=MAX_INPUT_LENGTH, max_summary_length=MAX_SUMMARY_LENGTH):
    inputs = [prefix + doc for doc in examples["text"]]
    
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=max_summary_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



In [ ]:
tokenizer('hi how are you', max_length=MAX_INPUT_LENGTH, truncation=True)

{'input_ids': [7102, 149, 33, 25, 1], 'attention_mask': [1, 1, 1, 1, 1]}

In [ ]:
def generate_encodings(sentences, instructions, tokenizer, max_input_length=MAX_INPUT_LENGTH, max_summary_length=MAX_SUMMARY_LENGTH):
    examples = []
    for sentence, instruction in zip(list(sentences), list(instructions)):
        try:            
            sentence = str(sentence)
            instruction = str(instruction)
            example = {'text': sentence, 'summary': instruction}
            model_inputs = tokenizer(sentence, max_length=max_input_length, truncation=True)
            labels = tokenizer(text_target=instruction, max_length=max_summary_length, truncation=True)
            model_inputs['labels'] = labels['input_ids']
            example['input_ids'] = model_inputs['input_ids']
            example['attention_mask'] = model_inputs['attention_mask']
            example['labels'] = model_inputs['labels']
            examples.append(example)
        except Exception as e:
            print(sentence, instruction)
            continue
    
    return examples

In [ ]:
class YouCookDatasetForKnowledgeExtraction(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        item = self.encodings[idx]
        item['input_ids'] = torch.tensor(item['input_ids'])
        item['attention_mask'] = torch.tensor(item['attention_mask'])
        item['labels'] = torch.tensor(item['labels'])
        return item

    def __len__(self):
        return len(self.encodings)

In [ ]:
train_encodings = generate_encodings(train_sentences, train_instructions, tokenizer)
val_encodings = generate_encodings(val_sentences, val_instructions, tokenizer)

train_dataset = YouCookDatasetForKnowledgeExtraction(train_encodings)
val_dataset = YouCookDatasetForKnowledgeExtraction(val_encodings)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
MODEL_SAVE_DIR, LOG_SAVE_DIR

(PosixPath('/content/gdrive/MyDrive/IFT6759/quick-recipe/models/youcook_t5_small'),
 PosixPath('/content/gdrive/MyDrive/IFT6759/quick-recipe/logs/youcook_t5_small'))

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=str(MODEL_SAVE_DIR),
    logging_dir=str(LOG_SAVE_DIR),
    logging_steps=10,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,4.389200,4.084556,0.227700,0.082500,0.215200,0.215100,16.138200
2,3.415200,2.953156,0.339500,0.144000,0.327400,0.327300,11.888000
3,3.146500,2.507384,0.437300,0.203000,0.429600,0.429200,7.621800
4,2.655300,2.350186,0.457500,0.216400,0.448600,0.448200,6.997200
5,2.513600,2.265012,0.475700,0.225100,0.466900,0.466200,6.585400
6,2.544100,2.212165,0.487100,0.236700,0.477400,0.477100,6.354800
7,2.292200,2.172910,0.500100,0.247400,0.489900,0.489900,6.208200
8,2.590300,2.153184,0.507700,0.254100,0.497200,0.497600,6.185800
9,2.641900,2.139508,0.509500,0.257000,0.499600,0.499600,6.086800
10,2.374700,2.134829,0.508600,0.257800,0.498900,0.498900,6.111100


<ipython-input-18-b4fc7bd9ae1e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['input_ids'] = torch.tensor(item['input_ids'])
<ipython-input-18-b4fc7bd9ae1e>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['attention_mask'] = torch.tensor(item['attention_mask'])
<ipython-input-18-b4fc7bd9ae1e>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(item['labels'])
<ipython-input-18-b4fc7bd9ae1e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or so

TrainOutput(global_step=400, training_loss=2.992163128852844, metrics={'train_runtime': 181.1589, 'train_samples_per_second': 137.89, 'train_steps_per_second': 2.208, 'total_flos': 687620411424768.0, 'train_loss': 2.992163128852844, 'epoch': 10.0})

## Inference and metrics calculation

In [ ]:
MODEL_CHECKPOINT_PATH = str(MODEL_SAVE_DIR / 'checkpoint-400')

In [ ]:
summarizer = pipeline("summarization", model=MODEL_CHECKPOINT_PATH)

In [ ]:
total_true_pos = 0
total_num_predicted = 0
total_num_gold = 0
text_list = []
summary_list = []
predictions_list = []
true_positive_list = []
num_predicted_list = []
num_gold_list = []

for index in range(len(val_dataset)):
  if (index+1) % 50 == 0:
    print(f"Processing example {index+1}")
  text = val_dataset[index]['text']
  summary = val_dataset[index]['summary']
  text_words = text.split(' ')
  summary_words = summary.split(' ')
  max_len = len(text_words)
  predictions = summarizer(text, min_length=3, max_length=max_len)
  predicted_words = set(predictions[0]['summary_text'].split(' '))
  # print("Text: ", text)
  # print("Predicted: ", predictions)
  # print("Actual: ", summary)
  true_pos = len(set(predicted_words) & set(summary_words))
  num_predicted = len(set(predicted_words))
  num_gold = len(set(summary_words))
  total_true_pos += true_pos
  total_num_predicted += num_predicted
  total_num_gold += num_gold
  text_list.append(text)
  summary_list.append(summary)
  predictions_list.append(predictions[0]['summary_text'])
  true_positive_list.append(true_pos)
  num_predicted_list.append(num_predicted)
  num_gold_list.append(num_gold)

<ipython-input-16-b4fc7bd9ae1e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['input_ids'] = torch.tensor(item['input_ids'])
<ipython-input-16-b4fc7bd9ae1e>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['attention_mask'] = torch.tensor(item['attention_mask'])
<ipython-input-16-b4fc7bd9ae1e>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(item['labels'])


Processing example 50
Processing example 100
Processing example 150
Processing example 200
Processing example 250
Processing example 300
Processing example 350
Processing example 400
Processing example 450
Processing example 500
Processing example 550
Processing example 600
Processing example 650
Processing example 700
Processing example 750
Processing example 800
Processing example 850
Processing example 900
Processing example 950
Processing example 1000
Processing example 1050


In [ ]:
_predictions = [prediction[0]['summary_text'] for prediction in predictions_list]
predictions_list = _predictions

In [ ]:
results_df = pd.DataFrame({'text': text_list,
                'summary': summary_list,
                'prediction': predictions_list,
                'true_positives': true_positive_list,
                'num_predicted': num_predicted_list,
                'num_gold': num_gold_list
                })

In [ ]:
results_df.to_csv(str(LOG_SAVE_DIR / 'val_performance.csv'), index=False)

In [ ]:
pd.read_csv(str(LOG_SAVE_DIR / 'val_performance.csv'))

,text,summary,prediction,true_positives,num_predicted,num_gold
0,"one whole egg, some dry italian herbs have a l...","take egg, Italian herb, granulated garlic, red...","mix garlic, red pepper, olive oil, olive oil, ...",2,8,9
1,"so when you 're slurries, mix set that aside a...","add black pepper, white pepper and salt","mix black pepper, white pepper, salt and pepper",6,7,7
2,i 'm gonna add some parmesan cheese.,add parmesan cheese,add parmesan cheese,3,3,3
3,you 're going to transfer your slurry mixture ...,transfer mixture,transfer mixture into meat bowl,2,5,2
4,i 'm going to stop and begin to shape the meat...,shape meatballs,meatballs,1,1,2
...,...,...,...,...,...,...
1066,"so of vegetable oil into a large, deep and hea...",oil skillet,add vegetable oil into skillet,2,5,2
1067,"best , if you have one as it holds, and distri...",fry at 365 degrees,heat oil to approximately three hundred and si...,1,11,4
1068,"the chicken pieces in the skillet and fry, the...",fry for 20 minutes,fry chicken pieces in skillet,1,5,4
1069,"pieces , several times during the cooking time...",drain chicken,drain chicken on paper towel,2,5,2


In [ ]:
precision = round(total_true_pos / total_num_predicted, 2)
recall = round(total_true_pos / total_num_gold, 2)
f1 = round(2*(precision*recall) / (precision+recall), 2)

print(f"Total true positives (predicted words overlap with gold words): {total_true_pos}")
print(f"Total predicted words: {total_num_predicted}")
print(f"Total gold words: {total_num_gold}")
print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")

Total true positives (predicted words overlap with gold words): 1722
Total predicted words: 5673
Total gold words: 3795
Precision: 0.3, Recall: 0.45, F1: 0.36


In [5]:
def best_keyclip_prediction_true_positives(row):
    return row['true_positives'] if row['IsPredUseful'] == 1 else 0

def generate_metrics_from_master_and_val_df(master_df, val_df):
  combined_df = pd.merge(master_df, val_df, left_on='Sentence', right_on='text', how='inner')
  combined_df['best_kc_pred_true_positives'] = combined_df.apply(lambda row: best_keyclip_prediction_true_positives(row), axis=1)
  total_num_gold = combined_df['num_gold'].sum()
  total_num_predicted = combined_df['num_predicted'].sum()
  total_true_pos = combined_df['best_kc_pred_true_positives'].sum()
  precision = round(total_true_pos / total_num_predicted, 2)
  recall = round(total_true_pos / total_num_gold, 2)
  f1 = round(2*(precision*recall) / (precision+recall), 2)

  print(f"Total true positives (predicted words overlap with gold words): {total_true_pos}")
  print(f"Total predicted words: {total_num_predicted}")
  print(f"Total gold words: {total_num_gold}")
  print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
  
  return precision, recall, f1

## Generate Knowledge Extraction metrics based on best Key-clip Prediction model's predictions

In [6]:
master_df = pd.read_pickle(DATA_DIR / 'full_master_updated.pkl')
val_df = pd.read_csv(str(LOG_SAVE_DIR / 'val_performance.csv'))

In [7]:
generate_metrics_from_master_and_val_df(master_df, val_df)

Total true positives (predicted words overlap with gold words): 1392
Total predicted words: 5404
Total gold words: 3644
Precision: 0.26, Recall: 0.38, F1: 0.31


(0.26, 0.38, 0.31)